# Interroger des documents (RAG)

In [ ]:
%pip install -qU wget openai

In [12]:
# OpenAI client configuration
import requests
from openai import OpenAI

base_url = "https://albert.api.etalab.gouv.fr/v1"
api_key = "YOUR_API_KEY"

client = OpenAI(base_url=base_url, api_key=api_key)

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}

Commençons par télécharger le document que nous souhaitons interroger. Ce document peut être un pdf, un fichier html ou un fichier json.


In [13]:
# Download a file
import wget
import os

file_path = "my_document.pdf"
if not os.path.exists(file_path):
    doc_url = "https://www.legifrance.gouv.fr/download/file/rxcTl0H4YnnzLkMLiP4x15qORfLSKk_h8QsSb2xnJ8Y=/JOE_TEXTE"

wget.download(doc_url, out=file_path)

'my_document (1).pdf'

Pour commencer, nous créons une collection nommée `tutorial`. Pour cela nous effectuons une requête GET sur l'endpoint `/v1/models` afin d'obtenir la liste des modèles disponibles et définissons le modèle d'embeddings à utiliser.

Nous allons avoir besoin également d'un modèle de langage. Nous appelons le endpoint `/v1/models` pour obtenir la liste des modèles. Les modèles de langage ont le type *text-generation* et les modèles d'embeddings le type *text-embeddings-inference*.


In [14]:
language_model, embeddings_model = None, None

for model in client.models.list().data:
    if model.type == "text-generation" and language_model is None:
        language_model = model.id
    if model.type == "text-embeddings-inference" and embeddings_model is None:
        embeddings_model = model.id

print(f"language model: {language_model}\nembeddings model: {embeddings_model}")

language model: AgentPublic/llama3-instruct-8b
embeddings model: BAAI/bge-m3


In [16]:
collection = "tutorial"

response = session.post(f"{base_url}/collections", json={"name": collection, "model": embeddings_model})
response = response.json()
collection_id = response["id"]
print(f"Collection ID: {collection_id}")

Collection ID: a8ce92d1-4b38-48b2-8459-7ebe9ae84c2f


Enfin pour nous importons le document dans la collection de notre base vectorielle à l'aide du endpoint POST `/v1/files`.

In [17]:
files = {"file": (os.path.basename(file_path), open(file_path, "rb"), "application/pdf")}
data = {"request": '{"collection": "%s"}' % collection_id}
response = session.post(f"{base_url}/files", data=data, files=files)
print(response.json())

{'id': '4ab9f9d1-fbf7-40a4-81f0-892bab407f1f'}


Nous pouvons observer que le fichier que nous avons importé est bien dans la collection à l'aide du endpoint GET `/v1/files`.

In [18]:
response = session.get(f"{base_url}/files/{collection_id}")
response.json()
files = response.json()["data"]
print(len(files))

1


Maintenant que nous avons notre collection et notre fichier, nous pouvons faire une recherche vectorielle à l'aide du endpoint POST `/v1/search`. Ces résutats de recherche vectorielle seront utilisés pour générer une réponse à l'aide du modèle de langage.

In [26]:
prompt = "Qui est Ulrich Tan ?"
data = {"collections": [collection_id], "k": 6, "prompt": prompt}
response = session.post(url=f"{base_url}/search", json=data, headers={"Authorization": f"Bearer {api_key}"})

prompt_template = "Réponds à la question suivante en te basant sur les documents ci-dessous : {prompt}\n\nDocuments :\n\n{chunks}"
chunks = "\n\n\n".join([result["chunk"]["content"] for result in response.json()["data"]])
sources = set([result["chunk"]["metadata"]["document_name"] for result in response.json()["data"]])
prompt = prompt_template.format(prompt=prompt, chunks=chunks)

response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=language_model,
    stream=False,
    n=1,
)

response = response.choices[0].message.content
print(response)

Selon les documents, M. Ulrich Tan est le chef du pôle Datamin du département "Eatalab".


Nous avons récupéré les sources depuis les metadata de chaque chunk. Nous pouvons constater que les chunks proviennent bien du document que nous avons importé.


In [27]:
print(sources)

{'my_document.pdf'}


Vous pouvez également faire ajouter une recherche sur internet en spécifiant "internet" dans la liste des collections.

In [31]:
prompt = "Qui est Ulrich Tan ?"
data = {"collections": ["internet"], "k": 6, "prompt": prompt}
response = session.post(url=f"{base_url}/search", json=data, headers={"Authorization": f"Bearer {api_key}"})

prompt_template = "Réponds à la question suivante en te basant sur les documents ci-dessous : {prompt}\n\nDocuments :\n\n{chunks}"
chunks = "\n\n\n".join([result["chunk"]["content"] for result in response.json()["data"]])
sources = set([result["chunk"]["metadata"]["document_name"] for result in response.json()["data"]])
prompt = prompt_template.format(prompt=prompt, chunks=chunks)

response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model=language_model,
    stream=False,
    n=1,
)

response = response.choices[0].message.content
print(response)

Ulrich Tan est le responsable de coordonner l'équipe du DataLab et d'accompagner les acteurs publics dans l'identification et la priorisation de cas d'usage d'intelligence artificielle pour leur administration.


Nous pouvons de la même manière récupérer les sources depuis les metadata de chaque chunk et les afficher.


In [32]:
print(sources)

{'https://www.etalab.gouv.fr/datalab/equipe/'}


Enfin il est possible de combiner une recherche dans les fichiers et une recherche sur internet.